In [103]:
import matplotlib.pyplot as plt
import json
import seaborn
from pathlib import Path
import numpy as np
from statistics import mean, median

In [104]:
def split_simulations_with_and_without_expansion(simulations):
    with_expansions = []
    without_expansions = []
    for simulations_per_model in simulations:
        for simulations_per_game in simulations_per_model:
            for simulations_per_move in simulations_per_game:
                with_expansions_for_move, without_expansions_for_move = simulations_per_move
                with_expansions.append(with_expansions_for_move)
                without_expansions.append(without_expansions_for_move)
    return with_expansions, without_expansions
            
game = "hex"
results = []
complex_simulations = []
simple_simulations = []
for result_file in Path(f"{game}/complex_rollouts/without_state_evaluator/").iterdir():
    with open(result_file) as f:
        result = json.load(f)
        results.append(result["results"])
        complex_simulations.append(result["average_complex_simulations"])
        simple_simulations.append(result["average_simple_simulations"])
data = np.average(results, axis=0)
complex_simulations_with_expansion, complex_simulations_without_expansion = split_simulations_with_and_without_expansion(complex_simulations)
simple_simulations_with_expansion, simple_simulations_without_expansion = split_simulations_with_and_without_expansion(simple_simulations)
average_complex_simulations = mean(complex_simulations_with_expansion), mean(complex_simulations_without_expansion)
average_simple_simulations = mean(simple_simulations_with_expansion), mean(simple_simulations_without_expansion)

In [105]:
print(np.average(results, axis=2).shape)
print(np.average(data, axis=1), np.sqrt(np.sum(np.std(np.average(results, axis=2), axis=0) ** 2, axis=0) / (20 ** 2)))
print(average_complex_simulations)
print(average_simple_simulations)

[0.3525 0.     0.6475] 0.01392813898392587
(72.78568469028382, 1585.0422622747049)
(543.7841792877609, 2259.0058329922226)
